# Bonus round

The data that has been provided were all extracted from the public Binance API. If you would like to demonstrate your ability to collect data from external data sources, feel free to provide code, along with your main submission, on how you would interact with the API to recreate the provided dataset.

- referenced to this repo https://github.com/nicknochnack/BinanceEDA

- due to time limit only candles data is fetched.

- funding rate: 
https://python-binance.readthedocs.io/en/latest/binance.html?highlight=funding%20rate#binance.client.Client.futures_funding_rate
https://binance-docs.github.io/apidocs/futures/en/#get-funding-rate-history

- open interest:
https://python-binance.readthedocs.io/en/latest/binance.html?highlight=open%20interest#binance.client.Client.futures_open_interest
https://binance-docs.github.io/apidocs/futures/en/#open-interest



In [1]:
api_key=""
secret_key=""

In [2]:
# install required packages

!pip install python-binance pandas

In [3]:
# import required packages

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.enums import HistoricalKlinesType
import pandas as pd

# Authenticate & fetch

In [4]:
client = Client(api_key, secret_key)

In [5]:
coins = ['binance-ADAUSDT-future', 'binance-BNBUSDT-future',
 'binance-BTCUSDT-future', 'binance-ETHUSDT-future',
 'binance-XRPUSDT-future']

In [6]:
# create dictionary to store candlesticks for different coins

candles = {}

In [7]:
# for each coin, fetch its historical data and tidy it in a df

for coin in coins:
    coin_name = coin.split('-')[1]
    # call api using package
    historical = client.get_historical_klines(coin_name, 
                                              interval=Client.KLINE_INTERVAL_5MINUTE, 
                                              limit=1000, 
                                              klines_type=HistoricalKlinesType.FUTURES,
                                              start_str="2021-06-01 00:00:00",
                                              end_str="2021-08-31 23:55:00"
                                             )
    hist_df = pd.DataFrame(historical)
    # name the columns (there are several that we will drop)
    hist_df.columns = ['datetime', 'price_open', 'price_high', 'price_low', 'price_close', 'volume', 'close_time', 'candle_usd_volume', 
                    'candle_trades_count', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    hist_df['datetime'] = pd.to_datetime(hist_df['datetime']/1000, unit='s')
    # calculate vwap
    hist_df = hist_df.astype({'candle_usd_volume':'float','volume':'float'})
    hist_df['vwap'] = hist_df['candle_usd_volume']/hist_df['volume']
    
    hist_df['market'] = coin
    candles[coin] = hist_df[['market',
                            'price_open',
                            'price_close',
                            'price_high',
                            'price_low',
                            'vwap',
                            'volume',
                            'candle_usd_volume',
                            'candle_trades_count',
                            'datetime']]
    

In [8]:
# combine all coins' df into one big df

all_candles = pd.concat(candles)

In [9]:
all_candles

market price_open price_close  \
binance-ADAUSDT-future 0      binance-ADAUSDT-future    1.73690     1.75790   
                       1      binance-ADAUSDT-future    1.75790     1.75650   
                       2      binance-ADAUSDT-future    1.75640     1.76590   
                       3      binance-ADAUSDT-future    1.76570     1.75860   
                       4      binance-ADAUSDT-future    1.75850     1.76110   
...                                              ...        ...         ...   
binance-XRPUSDT-future 26491  binance-XRPUSDT-future     1.1847      1.1819   
                       26492  binance-XRPUSDT-future     1.1818      1.1936   
                       26493  binance-XRPUSDT-future     1.1937      1.1910   
                       26494  binance-XRPUSDT-future     1.1910      1.1989   
                       26495  binance-XRPUSDT-future     1.1990      1.1872   

                             price_high price_low      vwap      volume  \
binance-ADAUSDT-future 0        1.75960   1.72430  1.745112  10911603.0   
                       1        1.76100   1.74960  1.756413   5233441.0   
                       2        1.76960   1.75550  1.765150   3983706.0   
                       3        1.77360   1.75820  1.766175   3956858.0   
                       4        1.76340   1.75090  1.757195   3302787.0   
...                                 ...       ...       ...         ...   
binance-XRPUSDT-future 26491     1.1849    1.1808  1.183065   2955184.1   
                       26492     1.1955    1.1806  1.188294   7993302.9   
                       26493     1.1951    1.1890  1.191987   4932346.3   
                       26494     1.2000    1.1903  1.195132   6157879.8   
                       26495     1.1991    1.1854  1.191439   9530010.3   

                              candle_usd_volume  candle_trades_count  \
binance-ADAUSDT-future 0           1.904196e+07                19737   
                       1           9.192086e+06                 9538   
                       2           7.031837e+06                 9721   
                       3           6.988504e+06                 8700   
                       4           5.803639e+06                 7094   
...                                         ...                  ...   
binance-XRPUSDT-future 26491       3.496173e+06                 4293   
                       26492       9.498396e+06                10281   
                       26493       5.879291e+06                 5815   
                       26494       7.359481e+06                 7772   
                       26495       1.135442e+07                 8056   

                                        datetime  
binance-ADAUSDT-future 0     2021-06-01 00:00:00  
                       1     2021-06-01 00:05:00  
                       2     2021-06-01 00:10:00  
                       3     2021-06-01 00:15:00  
                       4     2021-06-01 00:20:00  
...                                          ...  
binance-XRPUSDT-future 26491 2021-08-31 23:35:00  
                       26492 2021-08-31 23:40:00  
                       26493 2021-08-31 23:45:00  
                       26494 2021-08-31 23:50:00  
                       26495 2021-08-31 23:55:00  

[132480 rows x 10 columns]

In [10]:
import os.path

if os.path.exists("bonus_dataset/"):
    pass
else:
    os.mkdir("bonus_dataset/")

all_candles.to_csv("bonus_dataset/candles-bonus.csv", index=False)